# climada III: Hazard and TropCyclone (using xarray and pint)

Prepared by G. Aznar Siguan

# Hazard class

A Hazard contains the following information:

- tag (TagHazard): information about the source
- units (str): units of the intensity
- centroids (Centroids): centroids of the events
- event_id (np.array): id (>0) of each event
- event_name (list(str)): name of each event (default: event_id)
- date (np.array): integer date corresponding to the proleptic Gregorian ordinal, where January 1 of year 1 has ordinal 1 (ordinal format of datetime library)
- frequency (np.array): frequency of each event in seconds
- orig_event (np.array): flags indicating historical events (True) or probabilistic (False)
- intensity (sparse.csr_matrix): intensity of the events at centroids
- fraction (sparse.csr_matrix): fraction of affected exposures for each event at each centroid

The `intensity` and `fraction` variables are sparse matrices of size num_events x num_centroids. The sparse matrix is an object defined in the scipy library. There are different types (row-wise, column-wise, linked-lists, ...) and functions that can be used with them: https://docs.scipy.org/doc/scipy/reference/sparse.html.

Like with the other climada classes, a `Hazard` can be filled from a file as follows:

In [ ]:
% matplotlib inline
from climada import Hazard, HAZ_DEMO_MAT
# Unlike in other climada classes, Hazard needs to know the acronym of the hazard type in first place!!!
haz_tc_fl = Hazard('TC', HAZ_DEMO_MAT, 'Historic and synthetic tropical cyclones in Florida from 1851 to 2011.')
print()
print('Absolute file path of file read:', haz_tc_fl.tag.file_name)
print()
print('Hazard info:')
print(haz_tc_fl)

It has three different plot functions: `plot_intensity()`, `plot_fraction()`and `plot_stats()`. Depending on the inputs, different properties can be visualized. Check the documentation of a function:

In [ ]:
help(haz_tc_fl.plot_intensity)
help(haz_tc_fl.plot_stats)

### EXERCICE

Plot:

1. intensities of the largest event
2. maximum intensities at each centroid
3. intensities of event HAZEL
4. tropical cyclone intensities maps for the return periods [10, 50, 75, 100]
5. intensities of all the events in centroid with id 50
6. intensities of all the events in centroid closest to lat, lon = (26.5, -81)

In [ ]:
# Put your code here





In [ ]:
# SOLUTION:

# 1. intensities of the largest event:
haz_tc_fl.plot_intensity(event=-1)

# 2. maximum intensities at each centroid:
haz_tc_fl.plot_intensity(event=0)

# 3. intensities of event HAZEL:
haz_tc_fl.plot_intensity(event='HAZEL', cmap='BuGn') # setting color map

# 4. tropical cyclone intensities maps for the return periods [10, 50, 75, 100]
_, _, res = haz_tc_fl.plot_stats([10, 50, 75, 100])

# 5. intensities of all the events in centroid with id 50
haz_tc_fl.plot_intensity(centr=50)

# 5. intensities of all the events in centroid closest to lat, lon = (26.5, -81)
haz_tc_fl.plot_intensity(centr=(26.5, -81));

The following methods can be used to analyse the data in `Hazard`:

- `calc_year_set()` method returns a dictionary with all the historical (not synthetic) event ids that happened at each year. 
- `get_date_strings()` returns strings of dates in ISO format.
- To obtain the relation between event ids and event names, two methods can be used `event_id_to_name()` and `event_name_to_id()`. 

Finally, the methods `clear()` and `append()` are used to reinitialize the data in the `Hazard` instance and to expand the data with data from an other `Hazard`, respectively.

### EXERCISE:

1. How many hurricanes with the name `HAZEL` have happened in Florida? When did they take place?
2. How many tropical cyclones occured in year 2000?

In [ ]:
# Put your code here:





In [ ]:
# SOLUTION:

# 1. How many hurricanes with the name `HAZEL` have happened in Florida? When did they take place?
print('Number of events HAZEL:', haz_tc_fl.event_name_to_id('HAZEL').size)
print('Occurence of events HAZEL:', haz_tc_fl.get_date_strings('HAZEL'))

# 2. Which is the year where more tropical cyclones have occured?
# First we compute the events per year
year_set = haz_tc_fl.calc_year_set()
print('Number of events in 2000:', year_set[2000].size)

# Tropical Cyclone tracks

The `trop_cyclone` module contains the definition of the `TropCyclone` class and functions to read and deal with tropical cyclone tracks of the [IBTrACS](https://www.ncdc.noaa.gov/ibtracs/) repository. Executing the help function on the module all the classes and public functions of the module are described.

In [ ]:
from climada import trop_cyclone
help(trop_cyclone)

The functions `read_ibtracs()`, `interp_track()`, `calc_random_walk()` and `gust_from_track()` deal directly with tracks, without having to deal with the `TropCyclone` class. At higher level, however, it's more convenient to let the `TropCyclone` class (explained [below](#TropCyclone-class)) to deal with all those functions automatically. 

A tropical cyclone track information is stored in a `Dataset` of the xarray library.

## On xarray

Xarray's `DataArrays` is xarray’s implementation of a labeled, multi-dimensional array. The data is described for every defined coordinate.

Example:

In [ ]:
import pandas as pd
import xarray as xr
import numpy as np

# Coordinates:
locs = ['IA', 'IL', 'IN'] # spatial
times = pd.date_range('2000-01-01', periods=4) # temporal

# Data
obs = np.random.rand(4, 3) # time x space

# DataArray definition
foo = xr.DataArray(obs, coords=[times, locs], dims=['time', 'space'])

print('Whole DataArray: \n', foo)
print()
print('Select time: \n', foo.time)
print()
print('The values of a DataArray ar numpy.arrays: \n', foo.values)

Xarray's `Dataset` is multi-dimensional equivalent of a pandas `DataFrame` (see previous [DataFrame](Hands-on_III.ipynb#On-pandas)'s short description). Here both the data and coordinates are dict-like containers of `DataArray`.

Example:

In [ ]:
# Coordinates:
times = pd.date_range('2014-09-06', periods=5, freq='12H') # temporal
lon = [[-99.83, -99.32], [-99.79, -99.23]] # spatial first dim
lat = [[42.25, 42.21], [42.63, 42.59]]     # spatial second dim

# Data
temp = 15 + 8 * np.random.randn(2, 2, 5)
precip = 10 * np.random.rand(2, 2, 5)

ds = xr.Dataset({'temperature': (['x', 'y', 'time'],  temp),
                 'precipitation': (['x', 'y', 'time'], precip)},
                 coords={'lon': (['x', 'y'], lon),
                         'lat': (['x', 'y'], lat),
                         'time': times},
                 attrs={'t_unit': 'C'})

print('Whole Dataset: \n', ds)
print()
print('Select temperature: \n', ds.temperature)
print()
print('Select longitude: \n', ds.lon)
print()
print('Select latitude array: \n', ds.lat.values)

The syntax used for querying or plotting Datasets is similar to the one used in DataFrames. 

Example:

In [ ]:
# Compute mean temperature and precipitation per day:
print('Daily mean temperature and precipitation: \n', ds.groupby('time.day').mean())
print()
# Plot temperature time series in coord (x,y) = (1,0)
ds.temperature.isel(x=1, y=0).plot()

More on xarray: http://xarray.pydata.org/en/stable/data-structures.html

## Back to Tropical Cyclone tracks

### EXERCISE

Read "trac_brb_test.csv" file containing a tropical cyclone track in Barbados. 


In [ ]:
from climada import DATA_DIR
import os
BRB_TRACK = os.path.join(DATA_DIR, 'test', "trac_brb_test.csv")

# Put your code here




In [ ]:
# SOLUTION:
from climada import DATA_DIR
import os
# 1.Read "trac_brb_test.csv" file containing a tropical cyclone track in Barbados. 
BRB_TRACK = os.path.join(DATA_DIR, 'test', "trac_brb_test.csv")
tc_track_brb = trop_cyclone.read_ibtracs(BRB_TRACK)
tc_track_brb

The dimensions defined in the tropical cyclone track are different to those of usual weather observations (as the data used in [On xarray](#On-xarray)). We don't have a fixed location (x, y) and different measures at different times, but only an observation which is taken at a different locations for each time. This is the reason why the tracks only have one dimension, the time. However, different coordinates can be still defined: lat and lon for every time.

### EXERCISE

Using the previous tropical cyclone track,

1. Which is the time frequency of the data?
2. Compute the maximum sustained wind for each day.
3. Linearly interpolate the Dataset's data to a frequency of 1 hour.

In [ ]:
# Put your code here





In [ ]:
# SOLUTION:
import numpy as np
# 1. Which is the time frequency of the data?
# Since there are few elements, one can see with the naked eye that the frequence is 6 hours
# It can also be computed:
# The values of a DataArray are numpy.arrays. 
# The nummpy.ediff1d computes the different between elements in an array
diff_time_ns = np.ediff1d(tc_track_brb.time)
# pint library would offer a more elegant solution, as shown below
diff_time_h = diff_time_ns.astype(int)/1000/1000/1000/60/60
print('Mean time frequency in hours:', diff_time_h.mean())
print('Std time frequency in hours:', diff_time_h.std())
print()

# 2. Compute the maximum sustained wind for each day.
print('Daily max sustained wind:', tc_track_brb.max_sustained_wind.groupby('time.day').max())
print()

# 3. Linearly interpolate the Dataset's data to a frequency of 1 hour.
# Most of Pandas DataFrame functionalities work for xarray Dataset. For example, the resample method:
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html   
print('Interpolated Dataset:')
tc_track_brb.resample(time='1H').interpolate('linear')

## On pint

Pint ia a package to define, operate and manipulate physical quantities: the product of a numerical value and a unit of measurement. 

Example: The change of unit from ns to hours performed in the previous exercise can be done with pint, avoiding to use hardcoded numbers in your code and defining all the variables for changes of units.

In [ ]:
from pint import UnitRegistry
ureg = UnitRegistry()

# Scalar change of unit
val_in_ns = 21600000000000 * ureg.ns # Define your quantity with magnitude and unit
val_in_h = val_in_ns.to(ureg.hour)   # Change to an other unit
print('Value in hours:', val_in_h)
print('Magnitude in hours:', val_in_h.magnitude)

# Array change of unit
array_ns = diff_time_ns.astype(int)*ureg.ns # Define your quantity with magnitude and unit
array_h = array_ns.to(ureg.hour).magnitude  # Change to an other unit and get the magnitude (without the unit)
print('Array in hours:', array_h)

More on pint: https://pint.readthedocs.io/en/latest/

# TropCyclone class

The `TropCyclone` class is a derived class of `Hazard`. As such, it contains all the attributes and methods of a `Hazard`. Additionally, it contains the `tracks` variable, which is used to fill the TC tracks, if provided. 

Currently it contains three additional methods: `set_from_track()`, `set_random_walk()` and `plot_tracks()`. The first is used to fill the `TropCyclone` attributes from a IBTrACS track file (in csv format). The second generates synthetic tracks from the historical ones and computes the new hazard events from these tracks. The third is used to plot the `tracks` variable.

When setting a tropical cyclone from a track, the centroids can be provided. If no centroids are provided, the global centroids `GLB_NatID_grid_0360as_adv_2.mat` are used (as explained in [Centroids class](Hands-on_III.ipynb#Centroids-class)). The method used to calculate the wind gusts (the hazard intensity) can also be configured. The default (and only implemented up to now) is the method of Holland 2008.

In [ ]:
from climada import TropCyclone
help(TropCyclone)

### EXERCISE

* Build a `TropCyclone` instance from the "test/trac_brb_test.csv" file. 
* Plot the track.
* Print the date of the event.

In [ ]:
import os
from climada import DATA_DIR
BRB_TRACK = os.path.join(DATA_DIR, 'test', "trac_brb_test.csv")
# Put your code here:



In [ ]:
# SOLUTION:
import os
from climada import DATA_DIR
BRB_TRACK = os.path.join(DATA_DIR, 'test', "trac_brb_test.csv")

# When setting a tropical cyclone from a track, this can not be done at contruction
tc_haz_brb = TropCyclone()
tc_haz_brb.set_from_tracks(BRB_TRACK) # Default global centroids are used, which makes it slower

# Plotting track
tc_haz_brb.plot_tracks()

# The start date of the events are stored in the Hazard.date variable (without time).
# To change from ordinal format to date, we can use the datetime package
import datetime as dt
print('Date:', tc_haz_brb.get_date_strings())
# Since we also have the track information, we can check it in the tracks Dataset
print('Date and time:', tc_haz_brb.tracks[0].time[0].values)

### EXERCISE

Generate an ensamble of 9 synthetic events from the "test/trac_brb_test.csv" track. Use the centroids of barbados in "test/centr_brb_test.mat". Plot them all.

In [ ]:
# Put your code here





In [ ]:
# SOLUTION:
import os
from climada import DATA_DIR, Centroids
BRB_CENT = DATA_DIR + '/test/centr_brb_test.mat'
BRB_TRACK = os.path.join(DATA_DIR, 'test', "trac_brb_test.csv")

cent_brb = Centroids(BRB_CENT)

tc_haz_brb = TropCyclone()
tc_haz_brb.set_from_tracks(BRB_TRACK, centroids=cent_brb)
tc_haz_brb.set_random_walk(ens_size=9, centroids=cent_brb)
# Check that the data is consistent
tc_haz_brb.check()
print('Total number of events:', tc_haz_brb.event_id.size)
print('Total number of tracks:', len(tc_haz_brb.tracks))
tc_haz_brb.plot_tracks()

### EXERCISE

Read the tropical cyclones in `HAZ_DEMO_MAT` using the TropCyclone class. Check how many historical events are contained.

In [ ]:
# Put your code here:





In [ ]:
# SOLUTION:
from climada import HAZ_DEMO_MAT
tc_haz = TropCyclone(HAZ_DEMO_MAT)
print('Number of historical events:', tc_haz.orig.nonzero()[0].size)